##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Média do modelo

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/addons/tutorials/average_optimizers_callback"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/addons/tutorials/average_optimizers_callback.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/addons/tutorials/average_optimizers_callback.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
      <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/addons/tutorials/average_optimizers_callback.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>


## Visão geral

Este notebook demonstra como usar o Otimizador Moving Average com o Model Average Checkpoint do pacote do TensorFlow Addons.


## Moving Average

> A vantagem do Moving Average é que ele é menos propenso às mudanças desenfreadas de perda ou à representação de dados irregulares no lote mais recente. Ele dá uma ideia suavizada e mais geral do treinamento do modelo até um ponto específico.

## Stochastic Average

> A média de peso estocástico (SWA) converge para uma óptima mais larga. Ao fazer isso, ela se parece com o ensembling geométrico. A SWA é um método simples que melhora o desempenho do modelo quando é usado como um wrapper em outros otimizadores e resultados de média a partir de diferentes pontos da trajetória do otimizador interno.

## Model Average Checkpoint

> `callbacks.ModelCheckpoint` não oferece a opção de salvar os pesos de média móvel no meio do treinamento, portanto, os Otimizadores Model Average exigem uma callback personalizada. Usando o parâmetro `update_weights`, `ModelAverageCheckpoint` permite o seguinte:

1. Atribuir os pesos de média móvel ao modelo e salvá-los.
2. Manter os pesos não médios antigos, mas usar os pesos médios no modelo salvo.

## Configuração

In [ ]:
!pip install -U tensorflow-addons

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa

In [ ]:
import numpy as np
import os

## Crie o modelo 

In [ ]:
def create_model(opt):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),                         
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer=opt,
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

    return model

## Prepare o dataset

In [ ]:
#Load Fashion MNIST dataset
train, test = tf.keras.datasets.fashion_mnist.load_data()

images, labels = train
images = images/255.0
labels = labels.astype(np.int32)

fmnist_train_ds = tf.data.Dataset.from_tensor_slices((images, labels))
fmnist_train_ds = fmnist_train_ds.shuffle(5000).batch(32)

test_images, test_labels = test

Vamos comparar três otimizadores:

- SGD sem wrapper
- SGD com Moving Average
- SGD com a média de peso estocástico

Confira o desempenho deles com o mesmo modelo.

In [ ]:
#Optimizers 
sgd = tf.keras.optimizers.SGD(0.01)
moving_avg_sgd = tfa.optimizers.MovingAverage(sgd)
stocastic_avg_sgd = tfa.optimizers.SWA(sgd)

Ambos os otimizadores `MovingAverage` e `StochasticAverage` usam `ModelAverageCheckpoint`.

In [ ]:
#Callback 
checkpoint_path = "./training/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_dir,
                                                 save_weights_only=True,
                                                 verbose=1)
avg_callback = tfa.callbacks.AverageModelCheckpoint(filepath=checkpoint_dir, 
                                                    update_weights=True)

## Treine o modelo


### Otimizador SGD Vanilla 

In [ ]:
#Build Model
model = create_model(sgd)

#Train the network
model.fit(fmnist_train_ds, epochs=5, callbacks=[cp_callback])

In [ ]:
#Evalute results
model.load_weights(checkpoint_dir)
loss, accuracy = model.evaluate(test_images, test_labels, batch_size=32, verbose=2)
print("Loss :", loss)
print("Accuracy :", accuracy)

### SGD com Moving Average

In [ ]:
#Build Model
model = create_model(moving_avg_sgd)

#Train the network
model.fit(fmnist_train_ds, epochs=5, callbacks=[avg_callback])

In [ ]:
#Evalute results
model.load_weights(checkpoint_dir)
loss, accuracy = model.evaluate(test_images, test_labels, batch_size=32, verbose=2)
print("Loss :", loss)
print("Accuracy :", accuracy)

### SGD com a média de peso estocástico 

In [ ]:
#Build Model
model = create_model(stocastic_avg_sgd)

#Train the network
model.fit(fmnist_train_ds, epochs=5, callbacks=[avg_callback])

In [ ]:
#Evalute results
model.load_weights(checkpoint_dir)
loss, accuracy = model.evaluate(test_images, test_labels, batch_size=32, verbose=2)
print("Loss :", loss)
print("Accuracy :", accuracy)